<a href="https://colab.research.google.com/github/Negus25/Demo/blob/main/real_shuffle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.keras.layers import Input,  Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAvgPool2D, Reshape, Permute, Lambda

In [3]:
import tensorflow as tf

In [52]:
class shuffle_block(tf.keras.Model): 
    """The shuffle block of ShuffleNet."""
    def __init__(self, num_channels, groups=1, strides=1):
        super().__init__()
        self.gconv1 = tf.keras.layers.Conv2D(num_channels , padding='same',
                                            kernel_size=1, groups=groups)
        
        self.dconv1 = tf.keras.layers.DepthwiseConv2D( kernel_size=3,strides=2,
                                            padding='same')
        if strides == 2 :
          self.gconv2 = tf.keras.layers.Conv2D(num_channels - num_channels , padding='same',
                                            kernel_size=1, groups=groups)
        else:
          self.gconv2 = tf.keras.layers.Conv2D(num_channels, padding='same',
                                            kernel_size=1, groups=groups)
        self.group=groups
        self.AVGpool = None
        if strides>1:
          self.AVGpool= tf.keras.layers.AvgPool2D(3, strides = strides, padding= 'same')

        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.bn3 = tf.keras.layers.BatchNormalization()

    def call(self, X):
        Y = tf.keras.activations.relu(self.bn1(self.gconv1(X)))
        Y = channel_shuffle(Y,self.group)
        Y = self.bn2(self.dconv1(Y))
        Y = self.bn3(self.gconv2(Y))
        print(Y.shape)

        if self.AVGpool is not None:
            X = self.AVGpool(X)
            print(X.shape)
            Y= Add()([Y, X])
        else:
          Y= Add()([X, Y])
        return tf.keras.activations.relu(Y)


In [53]:
blk = shuffle_block(6, groups=3, strides = 2)
X = tf.random.uniform((4, 6, 6, 3))
Y = blk(X)
Y.shape

(4, 3, 3, 0)
(4, 3, 3, 3)


ValueError: ignored

In [6]:
def channel_shuffle(x, groups):
  _, width, height, channels = x.get_shape().as_list()
  group_ch = channels // groups
  x = Reshape([width, height, group_ch, groups])(x)
  x = Permute([1,2,4,3])(x)
  x = Reshape([width, height, channels])(x)
  return x


In [8]:

X = tf.random.uniform(((1, 28, 28, 384)))
Y = channel_shuffle(X, 8)
Y.shape

TensorShape([1, 28, 28, 384])

In [40]:
class NetBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels, repetitions, groups, first_block=False, **kwargs):
        super(NetBlock, self).__init__(**kwargs)
        self.shuffle_layers = []
        self.shuffle_layers.append(
            shuffle_block(num_channels, groups = groups, strides = 2))

        print(num_channels)
        for _ in range(repetitions):
            self.shuffle_layers.append(
                    shuffle_block(num_channels, groups = groups, strides=1))
       
           

    def call(self, X):
        for layer in self.shuffle_layers.layers:
            X = layer(X)
        return X

In [41]:
b1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(24, kernel_size=3, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])
b2 = NetBlock(384, 3, 8, first_block= True)
b3 = NetBlock(768, 7, 8, first_block= True)
b4 = NetBlock(1536, 3, 8)

384
768
1536


In [42]:
def net():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(24, kernel_size=3, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'), b2,
        tf.keras.layers.GlobalAvgPool2D(), tf.keras.layers.Dense(units=1000, activation= 'softmax')])

In [43]:
X = tf.random.uniform(shape=(1, 224, 224, 3))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2D output shape:	 (1, 112, 112, 24)
BatchNormalization output shape:	 (1, 112, 112, 24)
Activation output shape:	 (1, 112, 112, 24)
MaxPooling2D output shape:	 (1, 56, 56, 24)
(1, 28, 28, 384)
(1, 28, 28, 24)


ValueError: ignored